In [ ]:
from IPython.display import HTML, display
from tabulate import tabulate
import json
import pypint
from tqdm.notebook import tqdm

## Phases and markers

From [2 - Markers](2%20-%20Markers.ipynb)

In [ ]:
markers = json.load(open("generated/markers.json"))
phases = list(markers.keys())

## Initial state

From [3 - Initial state](3%20-%20Initial%20state.ipynb)

In [ ]:
inis = json.load(open("generated/inis.json"))

## Dynamical models

In [ ]:
ans = {code: pypint.load(f"generated/merge-{code}.an") for code in tqdm(inis)}

## B. Reach each phase with other phases to zero

In [ ]:
results = []
codes = list(sorted(inis.keys()))
for code in tqdm(codes):
    res_ini = [code]
    an = ans[code]
    for phase in tqdm(phases, desc=code, leave=bool(code == codes[0])):
        if phase == "M":
            continue # computed by another program
        res = []
        for marker in tqdm(markers[phase], desc=phase, leave=False):
            goal = {marker: 1}
            for p in phases:
                if p == phase:
                    continue
                for m in markers[p]:
                    goal[m] = 0

            reach = an.reachability(goal, reduce_for_goal=True, tool="mole")
            res.append(reach)
        res_ini.append("{}/{}".format(sum(res), len(res)))
    results.append(res_ini)
results = sorted(results, key=lambda x: x[0])

In [ ]:
display(HTML(tabulate(results, headers=["Precursor state"] + list(markers.keys()), tablefmt="html")))